In [1]:
from sqlalchemy import create_engine

# Fill in your actual MySQL username and password below
db_user = 'root'              # or your MySQL username
db_password = 'Bookbook@26'  # 🔒 Enter your MySQL password
db_host = 'localhost'
db_port = '3306'
db_name = 'sakila'

# Create engine
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Test connection
try:
    connection = engine.connect()
    print("✅ Connected successfully to the Sakila database!")
    connection.close()
except Exception as e:
    print("❌ Connection failed:", e)


❌ Connection failed: (mysql.connector.errors.DatabaseError) 2005 (HY000): Unknown MySQL server host '26@localhost' (11003)
(Background on this error at: https://sqlalche.me/e/20/4xp6)


In [2]:
from sqlalchemy import create_engine

# Use URL-encoded password (replace '@' with '%40')
db_user = 'root'
db_password = 'Bookbook%4026'   # '@' becomes '%40'
db_host = 'localhost'
db_port = '3306'
db_name = 'sakila'

# Create engine
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Test connection
try:
    connection = engine.connect()
    print("✅ Connected successfully to the Sakila database!")
    connection.close()
except Exception as e:
    print("❌ Connection failed:", e)


✅ Connected successfully to the Sakila database!


In [3]:
import pandas as pd

def rentals_month(engine, month, year):
    query = f"""
    SELECT *
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    return pd.read_sql(query, engine)


In [4]:
may_data = rentals_month(engine, 5, 2005)
may_data.head()


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [5]:
def rental_count_month(df, month, year):
    rental_count = df.groupby("customer_id").size().reset_index(name=f"rentals_{month:02d}_{year}")
    return rental_count


In [6]:
may_count = rental_count_month(may_data, 5, 2005)
may_count.head()


,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3


In [7]:
def compare_rentals(df1, df2):
    result = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    cols = result.columns
    result["difference"] = result[cols[2]] - result[cols[1]]
    return result


In [8]:
june_data = rentals_month(engine, 6, 2005)
june_count = rental_count_month(june_data, 6, 2005)

comparison = compare_rentals(may_count, june_count)
comparison.head()


,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,2.0
3,4,0.0,6.0,6.0
4,5,3.0,5.0,2.0
